In [87]:
import pandas as pd
import numpy as np
import os
import datetime

import set_config
import BATCHACTIVE
import QlikSense_OEE
import ORDER
import CNTR

# call set_config
dir_sanofi_share = set_config.ConfigSectionMap("SectionOne")['sanofi']
dir_local = set_config.ConfigSectionMap("SectionOne")['local']

# read IP21 SPEED files for AL6 packaging machines
folder = 'IP21_data'

Files = []
df = []



## FUNCTIONS

In [88]:
def create_df_from_file(indir, infolder, infilesearch):
    folder = infolder
    path = os.path.join(indir, folder)


    df = []
    df_created = pd.DataFrame()

    # do a recursive search for the files now we have multiple dirs
    for filename in iglob(path + '/**/*' + infilesearch + '*', recursive=True):
        df = pd.read_csv(filename)
        df_created = df_created.append(df)

    return df_created


def create_shift_category(passed_df):

    df = []
    df = passed_df
    
    bins = [0,5,13,21,24]


    labels = ['NIGHT','EARLY','LATE','NIGHT']

    df['Shift'] = pd.cut(df.index.hour,bins=bins, include_lowest=True, labels=labels, ordered=False)

    return df


# CHANGEOVER

In [89]:
# call changeover - calculated from the BATCHACTIVE files
df_changeover = BATCHACTIVE.changeover()

# ORDER

In [90]:
# get the batch info 
df_order = ORDER.ORDER_files()

# COUNTER

In [91]:
df_CNTR = CNTR.CNTR_Files()


C:\Users\mark_\anaconda3\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
C:\Users\mark_\anaconda3\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


# Merge Dataframes

## Create changeover category

In [95]:
df_CNTR.set_index('IP_TREND_TIME', inplace=True)

KeyError: "None of ['IP_TREND_TIME'] are in the columns"

In [93]:
# create a category of In_Changeover if times are between changeover start and end
for i, times in df_changeover.iterrows():
    df_CNTR.loc[(df_CNTR.index > times[0]) & (df_CNTR.index < times[1]), 'In_Changeover'] = 1

In [94]:
df_CNTR[df_CNTR['In_Changeover'] == 1]

,Name,IP_TREND_VALUE,IP_TREND_QLEVEL,IP_TREND_QSTATUS,Machine,Type,counter_diff,time_diff,In_Changeover
IP_TREND_TIME,,,,,,,,,
2021-08-02 22:53:40.300,36630901_CNTRB,29,Good,Good,Assembly,Bad,0.0,599.9,1.0
2021-08-02 23:03:39.900,36630901_CNTRB,29,Good,Good,Assembly,Bad,0.0,599.6,1.0
2021-08-02 23:09:32.200,36630901_CNTRB,30,Good,Good,Assembly,Bad,1.0,352.3,1.0
2021-08-02 23:09:41.300,36630901_CNTRB,31,Good,Good,Assembly,Bad,1.0,9.1,1.0
2021-08-02 23:09:47.400,36630901_CNTRB,32,Good,Good,Assembly,Bad,1.0,6.1,1.0
...,...,...,...,...,...,...,...,...,...
2021-09-30 14:42:50.700,36680902_FAULT,0,Good,Good,Palletizer,Bad,0.0,599.6,1.0
2021-09-30 14:52:49.900,36680902_FAULT,0,Good,Good,Palletizer,Bad,0.0,599.2,1.0
2021-09-30 15:02:49.100,36680902_FAULT,0,Good,Good,Palletizer,Bad,0.0,599.2,1.0


## Create Order Number column

Find rows for 36630901 order number changes and merge with df_CNTR

In [96]:
df_orders2 = df_order[df_order['36630901_ORDERNUMBER'].isna()==False]
# pd.DataFrame({'start':df_orders2.iloc[::2].values, 'end':df_orders2.iloc[1::2].values, 'time_diff_mins':df_orders2.time_diff_mins.iloc[1::2].values})
df_orders3 = pd.merge_asof(df_CNTR.sort_index(), df_orders2['36630901_ORDERNUMBER'].sort_index(), left_index=True, right_index=True, direction='backward', allow_exact_matches=True)


## Create Shift Category

In [97]:
# call create_shift_category with dataframe and which column you want to use
df_orders3 = create_shift_category(df_orders3)

In [98]:
df_orders3


,Name,IP_TREND_VALUE,IP_TREND_QLEVEL,IP_TREND_QSTATUS,Machine,Type,counter_diff,time_diff,In_Changeover,36630901_ORDERNUMBER,Shift
IP_TREND_TIME,,,,,,,,,,,
2021-08-01 00:02:30.500,36650901_CNTR_BAD_FS_F2,30,Good,Good,Packaging,Bad,NaN,NaN,NaN,80763298.0,NIGHT
2021-08-01 00:03:22.800,36640901_FAULT,0,Good,Good,Labeler,Bad,NaN,NaN,NaN,80763298.0,NIGHT
2021-08-01 00:04:08.100,36670901_FAULT,0,Good,Good,Packaging2,Bad,NaN,NaN,NaN,80763298.0,NIGHT
2021-08-01 00:06:25.800,36680902_FAULT,0,Good,Good,Palletizer,Bad,NaN,NaN,NaN,80763298.0,NIGHT
2021-08-01 00:06:42.200,36670901_CNTR_GOOD,29926,Good,Good,Packaging2,Good,NaN,NaN,NaN,80763298.0,NIGHT
...,...,...,...,...,...,...,...,...,...,...,...
2021-09-30 23:59:57.200,36640901_CNTRG,100756,Good,Good,Labeler,Good,3.0,1.0,NaN,80763298.0,NIGHT
2021-09-30 23:59:58.200,36630901_CNTR_GOOD,102227,Good,Good,Assembly,Good,8.0,3.1,NaN,80763298.0,NIGHT
2021-09-30 23:59:58.200,36640901_CNTRG,100761,Good,Good,Labeler,Good,5.0,1.0,NaN,80763298.0,NIGHT


In [104]:
df_orders3[df_orders3['In_Changeover'] == 1]

,Name,IP_TREND_VALUE,IP_TREND_QLEVEL,IP_TREND_QSTATUS,Machine,Type,counter_diff,time_diff,In_Changeover,36630901_ORDERNUMBER,Shift
IP_TREND_TIME,,,,,,,,,,,
2021-08-02 22:47:05.100,36650901_CNTR_BAD_FS_F2,0,Good,Good,Packaging,Bad,0.0,347.2,1.0,80763298.0,NIGHT
2021-08-02 22:47:05.100,36650901_CNTR_OUTPUT2,0,Good,Good,Packaging,Good,0.0,335.0,1.0,80763298.0,NIGHT
2021-08-02 22:48:21.600,36640901_CNTRG,225558,Good,Good,Labeler,Good,0.0,599.9,1.0,80763298.0,NIGHT
2021-08-02 22:48:34.800,36640901_CNTRB,165,Good,Good,Labeler,Bad,0.0,599.8,1.0,80763298.0,NIGHT
2021-08-02 22:51:15.600,36680902_FAULT,-1,Good,Good,Palletizer,Bad,0.0,599.8,1.0,80763298.0,NIGHT
...,...,...,...,...,...,...,...,...,...,...,...
2021-09-30 15:23:13.700,36640901_CNTRG,1941,Good,Good,Labeler,Good,6.0,1.0,1.0,80763298.0,LATE
2021-09-30 15:23:14.700,36630901_CNTR_GOOD,2866,Good,Good,Assembly,Good,2.0,2.0,1.0,80763298.0,LATE
2021-09-30 15:23:14.700,36640901_CNTRG,1946,Good,Good,Labeler,Good,5.0,1.0,1.0,80763298.0,LATE
